# Cargo las librerias y los archivos necesarios

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
ruta = '/Users/bita/Downloads/vitality/Data_Entry_2017.csv'

In [ ]:
df = pd.read_csv(ruta)
all_image_paths = {os.path.basename(x): x for x in glob(os.path.join('/Users/bita/Downloads/vitality/images/*.png'))}
df['path'] = df['Image Index'].map(all_image_paths.get)

# Selecciono solo las filas que tienen su imagen

In [ ]:
mask = df['path'].notnull()
df = df[mask]

In [ ]:
df = df[['path', 'Finding Labels']]

In [ ]:
import pandas as pd
import numpy as np
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array



mlb = MultiLabelBinarizer()
paths = df['path'].values
labels = df['Finding Labels'].str.split('|').values

In [ ]:
encoded_labels = mlb.fit_transform(labels)

In [ ]:

# Crear listas para almacenar las imágenes y las etiquetas procesadas
images = []
processed_labels = []

# Recorrer las rutas de las imágenes
for i,path in enumerate(list(paths)):
    # Cargar la imagen usando OpenCV
    image = cv2.imread(path)
    
    # Preprocesar la imagen (ajustar tamaño, normalizar, etc.)
    # Aquí puedes aplicar cualquier preprocesamiento específico que necesites
    # Por ejemplo, puedes redimensionar la imagen a un tamaño específico:
    image = cv2.resize(image, (224, 224), )
    image = img_to_array(image)
    image = image / 255.0  # Normalizar los valores de píxel
    
    # Agregar la imagen y las etiquetas procesadas a las listas
    images.append(image)
    processed_labels.append(encoded_labels[i])

# Convertir las listas en arrays de NumPy
images = np.array(images)
processed_labels = np.array(processed_labels)


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Tamaño de las imágenes
input_size = (224, 224, 3)

# Crear el modelo de CNN
model = Sequential()

# Agregar capas convolucionales y de pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_size))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Aplanar los mapas de características
model.add(Flatten())

# Agregar capas densas para la clasificación
model.add(Dense(64, activation='relu'))
model.add(Dense(15, activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Imprimir un resumen del modelo
model.summary()


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Tamaño de las imágenes
input_size = (224, 224, 3)

# Dividir los datos en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(images, processed_labels, test_size=0.2, random_state=42)


# Crear un nuevo modelo de Sequential
model = Sequential()

# Agregar el modelo pre-entrenado como capa
model.add(vgg16)

# Agregar capas adicionales para la clasificación
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(15, activation='softmax'))

# Compilar y entrenar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

